# Hitchhiker's Guide to Delta Lake (.NET for Spark C#)

This tutorial has been adapted for more clarity from its original counterpart [here](https://docs.delta.io/latest/quick-start.html). This notebook helps you quickly explore the main features of [Delta Lake](https://github.com/delta-io/delta). It provides code snippets that show how to read from and write to Delta Lake tables from interactive, batch, and streaming queries.

Here's what we will cover:
* Create a table
* Understanding meta-data
* Read data
* Update table data
* Overwrite table data
* Conditional update without overwrite
* Read older versions of data using Time Travel
* Write a stream of data to a table
* Read a stream of changes from a table

## Configuration
Make sure you modify this as appropriate.

In [27]:
var sessionId = (new Random()).Next(10000000);
var deltaTablePath = $"/delta/delta-table-{sessionId}";

deltaTablePath

/delta/delta-table-4339968

## Create a table
To create a Delta Lake table, write a DataFrame out in the **delta** format. You can use existing Spark SQL code and change the format from parquet, csv, json, and so on, to delta.

These operations create a new Delta Lake table using the schema that was inferred from your DataFrame. For the full set of options available when you create a new Delta Lake table, see Create a table and Write to a table (subsequent cells in this notebook).

In [28]:
var data = spark.Range(0,5);
data.Show();
data.Write().Format("delta").Save(deltaTablePath);

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

## Understanding Meta-data

In Delta Lake, meta-data is no different from data i.e., it is stored next to the data. Therefore, an interesting side-effect here is that you can peek into meta-data using regular Spark APIs. 

In [29]:
using System.Linq;
spark.Read().Text($"{deltaTablePath}/_delta_log/").Collect().ToList().ForEach(x => Console.WriteLine(x.GetAs<string>("value")));

{"commitInfo":{"timestamp":1604887209838,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"},"isBlindAppend":true,"operationMetrics":{"numFiles":"6","numOutputBytes":"2407","numOutputRows":"5"}}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"c1e86e66-d6b6-4fc0-8ab8-bf906e74fccf","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1604887208575}}
{"add":{"path":"part-00000-541921aa-d731-4e1b-a49e-69114154574b-c000.snappy.parquet","partitionValues":{},"size":262,"modificationTime":1604887209000,"dataChange":true}}
{"add":{"path":"part-00003-98f8511d-d355-4410-873d-b951d24e4e68-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1604887209000,"dataChange":true}}
{"add":{"path":"part-00006-1c72f731-9bd9-42b8-a98c-314857aabb7c-c000.snappy.pa

## Read data

You read data in your Delta Lake table by specifying the path to the files.

In [30]:
var df = spark.Read().Format("delta").Load(deltaTablePath);
df.Show()

+---+
| id|
+---+
|  0|
|  3|
|  2|
|  4|
|  1|
+---+

## Update table data

Delta Lake supports several operations to modify tables using standard DataFrame APIs. This example runs a batch job to overwrite the data in the table.


In [31]:
var data = spark.Range(5,10);
data.Write().Format("delta").Mode("overwrite").Save(deltaTablePath);
df.Show();

+---+
| id|
+---+
|  9|
|  5|
|  7|
|  6|
|  8|
+---+

When you now inspect the meta-data, what you will notice is that the original data is over-written. Well, not in a true sense but appropriate entries are added to Delta's transaction log so it can provide an "illusion" that the original data was deleted. We can verify this by re-inspecting the meta-data. You will see several entries indicating reference removal to the original data.

In [32]:
spark.Read().Text($"{deltaTablePath}/_delta_log/").Collect().ToList().ForEach(x => Console.WriteLine(x.GetAs<string>("value")));

{"commitInfo":{"timestamp":1604887222146,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"readVersion":0,"isBlindAppend":false,"operationMetrics":{"numFiles":"6","numOutputBytes":"2407","numOutputRows":"5"}}}
{"add":{"path":"part-00000-e5e5f4d5-aea5-4630-bc2b-f79d0bcfbe6f-c000.snappy.parquet","partitionValues":{},"size":262,"modificationTime":1604887221000,"dataChange":true}}
{"add":{"path":"part-00003-feddbddc-3c95-45ff-9597-75b2d0f61fd9-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1604887221000,"dataChange":true}}
{"add":{"path":"part-00006-071155a4-69a8-43ff-b4bd-2a97343c165a-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1604887221000,"dataChange":true}}
{"add":{"path":"part-00009-f9be07aa-7998-41a4-8f62-bca74739fe47-c000.snappy.parquet","partitionValues":{},"size":429,"modificationTime":1604887221000,"dataChange":true}}
{"add":{"path":"part-00012-6fe0f2c9-8792-4da2-834b-c8b2608fedd2-c000.snappy.p

## Save as catalog tables

Delta Lake can write to managed or external catalog tables.


In [33]:
// Write data to a new managed catalog table.
data.Write().Format("delta").SaveAsTable("ManagedDeltaTable");

In [34]:
// Define an external catalog table that points to the existing Delta Lake data in storage.
spark.Sql($"CREATE TABLE ExternalDeltaTable USING DELTA LOCATION '{deltaTablePath}'");

In [35]:
// List the 2 new tables.
spark.Sql("SHOW TABLES").Show();

// Explore their properties.
spark.Sql("DESCRIBE EXTENDED ManagedDeltaTable").Show(truncate: 0);
spark.Sql("DESCRIBE EXTENDED ExternalDeltaTable").Show(truncate: 0);

+--------+------------------+-----------+
|database|         tableName|isTemporary|
+--------+------------------+-----------+
| default|externaldeltatable|      false|
| default| manageddeltatable|      false|
+--------+------------------+-----------+

+----------------------------+-------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                              |comment|
+----------------------------+-------------------------------------------------------------------------------------------------------+-------+
|id                          |bigint                                                                                                 |null   |
|                            |                                                                                                       |       |
|# Detailed Table Information|  

## Conditional update without overwrite

Delta Lake provides programmatic APIs to conditional update, delete, and merge (upsert) data into tables. For more information on these operations, see [Table Deletes, Updates, and Merges](https://docs.delta.io/latest/delta-update.html).

In [36]:
using Microsoft.Spark.Extensions.Delta;
using Microsoft.Spark.Extensions.Delta.Tables;
using Microsoft.Spark.Sql;
using static Microsoft.Spark.Sql.Functions;

var deltaTable = DeltaTable.ForPath(deltaTablePath);

In [37]:
// Update every even value by adding 100 to it
deltaTable.Update(
  condition: Expr("id % 2 == 0"),
  set: new Dictionary<string, Column>(){{ "id", Expr("id + 100") }});
deltaTable.ToDF().Show();

+---+
| id|
+---+
|108|
|  9|
|  5|
|  7|
|106|
+---+

In [38]:
// Delete every even value
deltaTable.Delete(condition: Expr("id % 2 == 0"));
deltaTable.ToDF().Show();

+---+
| id|
+---+
|  9|
|  5|
|  7|
+---+

In [39]:
// Upsert (merge) new data
var newData = spark.Range(20).As("newData");

deltaTable
    .As("oldData")
    .Merge(newData, "oldData.id = newData.id")
    .WhenMatched()
        .Update(new Dictionary<string, Column>() {{"id", Lit("-1")}})
    .WhenNotMatched()
        .Insert(new Dictionary<string, Column>() {{"id", Col("newData.id")}})
    .Execute();

deltaTable.ToDF().Show(100);

+---+
| id|
+---+
|  0|
|  1|
| -1|
| 11|
| 17|
| -1|
| 18|
|  3|
| 14|
| 19|
| 10|
|  4|
| -1|
| 16|
|  2|
|  6|
| 12|
|  8|
| 15|
| 13|
+---+

## History
Delta's most powerful feature is the ability to allow looking into history i.e., the changes that were made to the underlying Delta Table. The cell below shows how simple it is to inspect the history.

In [40]:
deltaTable.History().Show(20, 1000, false);

+-------+-------------------+------+--------+---------+-------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|          timestamp|userId|userName|operation|                                                operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|                                                                                                                                                                                              operationMetrics|
+-------+-------------------+------+--------+---------+-------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------

## Read older versions of data using Time Travel

You can query previous snapshots of your Delta Lake table by using a feature called Time Travel. If you want to access the data that you overwrote, you can query a snapshot of the table before you overwrote the first set of data using the versionAsOf option.

Once you run the cell below, you should see the first set of data, from before you overwrote it. Time Travel is an extremely powerful feature that takes advantage of the power of the Delta Lake transaction log to access data that is no longer in the table. Removing the version 0 option (or specifying version 1) would let you see the newer data again. For more information, see [Query an older snapshot of a table (time travel)](https://docs.delta.io/latest/delta-batch.html#deltatimetravel).

In [41]:
var df = spark.Read().Format("delta").Option("versionAsOf", 0).Load(deltaTablePath);
df.Show();

+---+
| id|
+---+
|  0|
|  3|
|  2|
|  4|
|  1|
+---+

## Write a stream of data to a table

You can also write to a Delta Lake table using Spark's Structured Streaming. The Delta Lake transaction log guarantees exactly-once processing, even when there are other streams or batch queries running concurrently against the table. By default, streams run in append mode, which adds new records to the table.

For more information about Delta Lake integration with Structured Streaming, see [Table Streaming Reads and Writes](https://docs.delta.io/latest/delta-streaming.html).

In the cells below, here's what we are doing:

1. *Cell 28* Setup a simple Spark Structured Streaming job to generate a sequence and make the job write into our Delta Table
2. *Cell 30* Show the newly appended data
3. *Cell 31* Inspect history
4. *Cell 32* Stop the structured streaming job
5. *Cell 33* Inspect history <-- You'll notice appends have stopped

In [42]:
var streamingDf = spark.ReadStream().Format("rate").Load();
var stream = streamingDf.SelectExpr("value as id").WriteStream().Format("delta").Option("checkpointLocation", $"/tmp/checkpoint-{sessionId}").Start(deltaTablePath);

## Read a stream of changes from a table

While the stream is writing to the Delta Lake table, you can also read from that table as streaming source. For example, you can start another streaming query that prints all the changes made to the Delta Lake table.

In [43]:
deltaTable.ToDF().Sort(Col("id").Desc()).Show(100);

+---+
| id|
+---+
| 19|
| 18|
| 17|
| 16|
| 15|
| 14|
| 13|
| 12|
| 11|
| 10|
|  8|
|  6|
|  4|
|  3|
|  2|
|  1|
|  0|
| -1|
| -1|
| -1|
+---+

In [44]:
deltaTable.History().Drop("userId", "userName", "job", "notebook", "clusterId", "isolationLevel", "isBlindAppend").Show(20, 1000, false);

+-------+-------------------+----------------+-------------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|          timestamp|       operation|                                                                  operationParameters|readVersion|                                                                                                                                                                                              operationMetrics|
+-------+-------------------+----------------+-------------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------

In [45]:
stream.Stop();

In [46]:
deltaTable.History().Drop("userId", "userName", "job", "notebook", "clusterId", "isolationLevel", "isBlindAppend").Show(100, 1000, false);

+-------+-------------------+----------------+-------------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|          timestamp|       operation|                                                                  operationParameters|readVersion|                                                                                                                                                                                              operationMetrics|
+-------+-------------------+----------------+-------------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------

## Compaction

If a Delta Table is growing too large, you can compact it by repartitioning into a smaller number of files.

The option `dataChange = false` is an optimization that tells Delta Lake to do the repartition without marking the underlying data as "modified". This ensures that any other concurrent operations (such as streaming reads/writes) aren't negatively impacted.


In [47]:
int partitionCount = 2;

spark.Read()
    .Format("delta")
    .Load(deltaTablePath)
    .Repartition(partitionCount)
    .Write()
    .Option("dataChange", "false")
    .Format("delta")
    .Mode("overwrite")
    .Save(deltaTablePath);

## Convert Parquet to Delta
You can do an in-place conversion from the Parquet format to Delta.

In [48]:
var parquetPath = $"/parquet/parquet-table-{sessionId}";

var data = spark.Range(0,5);
data.Write().Parquet(parquetPath);

// Confirm that the data isn't in the Delta format
DeltaTable.IsDeltaTable(parquetPath)

False

In [49]:
DeltaTable.ConvertToDelta(spark, $"parquet.`{parquetPath}`");

//Confirm that the converted data is now in the Delta format
DeltaTable.IsDeltaTable(parquetPath)

True

## SQL Support
Delta supports table utility commands through SQL.  You can use SQL to:
* Get a DeltaTable's history
* Vacuum a DeltaTable
* Convert a Parquet file to Delta


In [50]:
spark.Sql($"DESCRIBE HISTORY delta.`{deltaTablePath}`").Show();

+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+
|version|          timestamp|userId|userName|       operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|
+-------+-------------------+------+--------+----------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+
|      6|2020-11-09 02:01:53|  null|    null|           WRITE|[mode -> Overwrit...|null|    null|     null|          5|          null|        false|[numFiles -> 2, n...|
|      5|2020-11-09 02:01:40|  null|    null|STREAMING UPDATE|[outputMode -> Ap...|null|    null|     null|          4|          null|         true|[numRemovedFiles ...|
|      4|2020-11-09 02:01:09|  null|    null|           MERGE|[predicate -> (ol...|null|    null|     null|          3|          null|        false|[n

In [51]:
spark.Sql($"VACUUM delta.`{deltaTablePath}`").Show();

+--------------------+
|                path|
+--------------------+
|abfss://data@...|
+--------------------+

In [52]:
var parquetId =  (new Random()).Next(10000000);
var parquetPath = $"/parquet/parquet-table-{sessionId}-{parquetId}";

var data = spark.Range(0,5);
data.Write().Parquet(parquetPath);

// Confirm that the data isn't in the Delta format
DeltaTable.IsDeltaTable(parquetPath);

// Use SQL to convert the parquet table to Delta
spark.Sql($"CONVERT TO DELTA parquet.`{parquetPath}`");

DeltaTable.IsDeltaTable(parquetPath);